In [ ]:
import shutil
from pathlib import Path

import ase
import ase.units
import matplotlib.pyplot as plt
import numpy as np
from ase.calculators.lj import LennardJones
from ase.md.langevin import Langevin
from tqdm import tqdm

import hillclimber as hc

In [ ]:
system = ase.Atoms(
    "Ar2",
    positions=[[0.0, 0.0, 0.0], [3.8, 0.0, 0.0]],
)

In [ ]:
epsilon_ar = 0.0104  # eV
sigma_ar = 3.4  # Angstrom
cutoff = 10.0  # Angstrom

lj_calc = LennardJones(sigma=sigma_ar, epsilon=epsilon_ar, rc=cutoff, smooth=True)

In [ ]:
atom1 = hc.IndexSelector(indices=[[0]])
atom2 = hc.IndexSelector(indices=[[1]])

distance_cv = hc.DistanceCV(x1=atom1, x2=atom2, prefix="dist", flatten=True)

# Step 4: Set up metadynamics
# Distance will vary around 3-5 Å, so use appropriate grid bounds
metad_bias = hc.MetadBias(
    cv=distance_cv,
    sigma=0.2,  # Width in Angstrom
    grid_min=0.0,
    grid_max=5.0,
)

wall = hc.UpperWallBias(
    cv=distance_cv,
    at=4,
    kappa=100.0,
)

metad_config = hc.MetaDynamicsConfig(
    height=0.005,  # Small height in eV
    pace=10,  # Deposit Gaussian every 10 steps
    temp=120.0,  # Temperature in Kelvin (Argon melts at ~84 K)
    biasfactor=10.0,  # Well-tempered metadynamics
    file="HILLS",
    flush=10,
)


# Step 5: Create minimal model class that provides the LJ calculator
class LJModel:
    """Minimal model for testing that wraps LJ calculator."""

    def get_calculator(self, directory=None, **kwargs):
        return lj_calc


# Step 6: Create MetaDynamicsModel
metad_model = hc.MetaDynamicsModel(
    config=metad_config,
    data=[system.copy()],
    data_idx=0,
    bias_cvs=[metad_bias],
    actions=[hc.PrintAction(cvs=[distance_cv], stride=5), wall],
    timestep=2.0,  # 2 fs timestep
    model=LJModel(),
)

In [ ]:
work_dir = Path("argon_metad")
work_dir.mkdir(exist_ok=True)

# Step 7: Get PLUMED-wrapped calculator
calc = metad_model.get_calculator(directory=work_dir)
system.calc = calc

In [ ]:
dyn = Langevin(
    atoms=system,
    timestep=2.0 * ase.units.fs,
    temperature_K=120.0,
    friction=0.01,  # 1/fs
    trajectory=str(work_dir / "md.traj"),
)

# Run 50 steps (100 fs total)
# With pace=10, this deposits 5 Gaussians
n_steps = 100_000
for _ in tqdm(dyn.irun(n_steps), total=n_steps):
    pass

In [ ]:
_ = hc.plot_cv_time_series("argon_metad/COLVAR")
hc.sum_hills(
    hills_file="argon_metad/HILLS",
    bin=500,
    outfile="argon_metad/fes.dat",
)

In [ ]:
data = np.loadtxt("argon_metad/fes.dat", comments="#")

# Extract columns
cv = data[:, 0]  # Collective variable (distance)
fes = data[:, 1]  # Free energy

_, (ax1) = plt.subplots(1, 1)

# Plot free energy surface
ax1.plot(cv, fes, "b-", linewidth=2)
ax1.set_ylabel("Free Energy (kJ/mol)", fontsize=12)
ax1.grid(True, alpha=0.3)

In [ ]:
shutil.rmtree(work_dir)